In [1]:
import pandas as pd
import numpy as np

In [18]:
df_base = pd.read_json("outputs/strategyqa-dev-modelcot.jsonl", orient='records', lines=True)
df_oracle_reversed = pd.read_json("outputs/gpt4-oracle-reversed.jsonl", orient='records', lines=True)
df_model_reversed = pd.read_json("outputs/gpt4-model-reversed.jsonl", orient='records', lines=True)
df_base.shape, df_oracle_reversed.shape, df_model_reversed.shape

((128, 9), (128, 5), (127, 5))

In [22]:
df = pd.concat(
    [
        df_base.assign(question_type = 'original'),
        df_oracle_reversed.rename(columns = lambda x: x.replace('reverse_','')).assign(question_type = 'oracle_reversed'),
        df_model_reversed.rename(columns = lambda x: x.replace('reverse_','')).assign(question_type = 'model_reversed')
    ]
)
df

,qid,term,description,question,answer,facts,decomposition,evidence,cot_answer,question_type
0,e0044a7b4d146d611e73,"Albany, Georgia","City in Georgia, United States",Will the Albany in Georgia reach a hundred tho...,False,"[Albany, GA has around 75,000 people, Albany, ...","[What is the population of Albany, Georgia?, W...","[[[['Albany, Georgia-1']], [['Albany, New York...",1. Albany in Georgia is smaller than Albany in...,original
1,c69397b4341b65ed080f,Saint Vincent and the Grenadines,Country in the Caribbean,Is the language used in Saint Vincent and the ...,True,[The primary language spoken in Saint Vincent ...,[What language is used in Saint Vincent and th...,[[[['Demographics of Saint Vincent and the Gre...,1. Saint Vincent and the Grenadines is a count...,original
2,1932e05f10680ece229f,Sea of Japan,"Marginal sea between Japan, Russia and Korea",Would the top of Mount Fuji stick out of the S...,True,"[The average depth of the Sea of Japan is 5,7...","[How tall is Mount Fuji?, What is the maximum ...","[[[['Mount Fuji-18']], [['Sea of Japan-15']], ...",1. Mount Fuji is located in Japan and not unde...,original
3,c91eafafed5a8f80bb5a,Miami,"City in Florida, United States",Is Miami a city on the American West Coast?,False,"[Miami is a city in the state of Florida., Flo...","[What state is Miami located in?, Which states...","[[[['Miami-1']], [['West Coast of the United S...",1. Miami is not located in the westernmost reg...,original
4,2047c0c34383f8014820,Swiss Guard,Military of Vatican City,Can the Swiss Guard fill the Virginia General ...,False,"[The Virginia General Assembly has 140 seats.,...","[What is the size of the Swiss Guard?, What is...","[[[['Swiss Guards-18'], no_evidence], [['Virgi...",1. The Swiss Guard is a military corps respons...,original
...,...,...,...,...,...,...,...,...,...,...
122,5f290c4202a54bf6aa71,Neil Armstrong,NaN,Did Neil Armstrong win the Space Race?,False,[Neil Armstrong was an astronaut and the first...,NaN,NaN,NaN,model_reversed
123,6e95d89ccd3256bde343,Bruno Mars,NaN,Did Bruno Mars always perform live perfectly?,True,[Bruno Mars is known for his disciplined appro...,NaN,NaN,NaN,model_reversed
124,7d1f1c2a9d554a017a22,Chicken,NaN,Are there bones in a chicken pizza?,False,[Chicken is a type of meat that often contains...,NaN,NaN,NaN,model_reversed
125,4070e4485b1bcc9e04d8,Garage,NaN,Can you hide a basketball in a garage?,False,[A basketball is a reasonably small object com...,NaN,NaN,NaN,model_reversed




oracle_facts, oracle_facts_reverse, model_facts, model_facts_reverse, oracle_answer, oracle_answer_reverse, model_answer, model_answer_reverse

oracle_edit = instills true answer, oracle_reverse_edit = cuts against true answer, model_edit = instills model answer, model_reverse_edit=cuts against model answer

test all by examining probability of answer

- oracle_edit = original facts
- model_edit = model's COT reasons
- oracle_reverse_edits = original subject gets associated with oracle_reverse_facts (facts from gpt4)
- model_reverse_edits = original subject gets associated with model_reverse_facts (facts from gpt4)

For new facts, create edit by taking reasoning and swapping [new_subject] --> [original_subject]

## Goal:

1. First, create a function to swap original and new subjects in the "reasoning".
2. Ultimately, we want to create a dataset of "edits". Each row will have the folllowing fields:

- qid 
- question
- edit_type [original-oracle, original-model, reversed-oracle, reversed-model]
- edit (looks like facts)
